In [3]:
"""Opportunistic Graph Search"""
import sys, os
import uuid
import numpy as np
import random
import math
import copy
import pandas as pd

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [4]:
class Order:
    def __init__(self, time_in: int, prep_time: int = 5):
        self.id = uuid.uuid4()
        self.time_in = time_in
        self.prep_time = prep_time
        self.coord = Order.random_coord(circle=False)
        self.is_out = False
    
    @staticmethod
    def random_coord(r=10, center=(0,0), circle=True):
        if circle:
            alpha = 2 * math.pi * random.random()
            
            # random radius
            r = r * math.sqrt(random.random())
            
            # calculating coordinates
            x = r * math.cos(alpha) + center[0]
            y = r * math.sin(alpha) + center[1]
            
            return (x, y)
        else:
            x = random.randrange(center[0],r)
            y = random.randrange(center[1], r)
            
            return (x, y)

In [5]:
class PathNode:
    def __init__(self, node_type: str , coord: (float, float) = (0,0), order_idx: int = 0):
        self.node_type = node_type
        self.coord = coord
        self.order_idx = order_idx
        self.arrival_time = float('inf')

In [6]:
class Path:
    def __init__(self, orders: [Order], route: [int], departure_time: float = float('inf'), previous_completion: float = 0, time: int = 0):
        self.departure_time = max(x.time_in + x.prep_time for x in [orders[x-1] for x in route[1:-1]])
        self.departure_time = max(time + 1, self.departure_time)
        #if previous_completion > 0:
        #    self.departure_time = self.departure_time if self.departure_time > previous_completion else previous_completion
    
        self.completion_time = float('inf')
        self.nodes = [PathNode('hub') if x == 0 else PathNode('order', orders[x - 1].coord, x - 1) for x in route]

In [7]:
p = Path(orders = [Order(58)], route = [0,1,0], previous_completion=81)

In [8]:
class Rider:
    def __init__(self, start_time: int, name: str):
        self.name = name
        self.start_time = start_time
        self.on_path = False # Waiting at Hub
        self.assigned_orders = []
        self.current_path: Path = None
        self.current_orders: [Order] = []
        self.next_path: Path = None
        self.next_orders: [Order] = []
        self.completed_paths = []
        self.completed_orders = []
        
    def update_path(self, orders: [Order], path: Path):
        # Rider is on path, modify the next
        if self.on_path:
            self.next_path = path
            self.next_orders = orders
        # Rider in not busy on path.
        else:
            self.current_path = path
            self.current_orders = orders

In [12]:
class RiderManager:
    def __init__(self, num_riders: int, time: int, verbose: bool = False):
        self.riders = [Rider(time, 'R{}'.format(x+1)) for x in range(num_riders)]
        self.orders = []
        self.orders_assigned = 0
        self.verbose = verbose
        
    # [ UTILS START ] 
    @staticmethod        
    def tsp_path(time_matrix):
        manager = pywrapcp.RoutingIndexManager(len(time_matrix), 1, 0) # Time_matrix, num_riders, depot
        routing = pywrapcp.RoutingModel(manager)
        
        # Time callback, gets the time it takes to travel between two nodes.
        def time_callback(from_index, to_index):
            # Convert from routing variable Index to time matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return time_matrix[from_node][to_node]
    
        transit_callback_index = routing.RegisterTransitCallback(time_callback)
        
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
        # Search parameters.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        
        # Solution.
        solution = routing.SolveWithParameters(search_parameters)
        
        # Routes from solution.
        path = []
        index = routing.Start(0)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        return route
    
    @staticmethod        
    def vrp_path(time_matrix, num_riders):
        manager = pywrapcp.RoutingIndexManager(len(time_matrix), num_riders, 0) # Time_matrix, num_riders, depot
        routing = pywrapcp.RoutingModel(manager)
        
        # Time callback, gets the time it takes to travel between two nodes.
        def time_callback(from_index, to_index):
            # Convert from routing variable Index to time matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return time_matrix[from_node][to_node]
    
        transit_callback_index = routing.RegisterTransitCallback(time_callback)
        
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        # Add Distance constraint.
        dimension_name = 'VRP-Time'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            120,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)
    
        # Search parameters.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        
        # Solution.
        solution = routing.SolveWithParameters(search_parameters)
        
        # Routes from solution.
        routes = []
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes
    
    @staticmethod
    def time_matrix(orders, handling_time=5) -> [[int]]:
        # From orders to nodes.
        nodes = [x.coord for x in orders]
        nodes.insert(0, (0,0))
        
        # Compute time matrix.
        tm = [[0 for x in range(len(nodes))] for y in range(len(nodes))]
        for row, start in enumerate(nodes):
            for col, end in enumerate(nodes):
                x = np.array(start)
                y = np.array(end)
                distance = round(np.linalg.norm(x - y)) # Round to nearest integer
            
                if (end == (0,0)) | (row == col):
                    tm[row][col] = distance 
                else:
                    tm[row][col] = distance + handling_time # We account for the handling time in the time to execute the path
       
        return tm
    
    def describe_paths(self, wait_time: int = 30):
        total_orders = 0
        for rider_idx, rider in enumerate(self.riders):
            print('Rider {}'.format(rider_idx + 1))
            print('Paths completed: {}\n'.format(len(rider.completed_paths)))
            for path_idx, path in enumerate(rider.completed_paths):
                print('Path {}'.format(path_idx + 1))
                print('Start: {}'.format(path.departure_time))
                print('End:   {}'.format(path.completion_time))
                print('Path time: {}'.format(path.completion_time - path.departure_time))
                print('Orders completed: {}\n'.format(len(path.nodes[1:-1])))
                
                for order_idx, order in enumerate(rider.completed_orders[path_idx]):
                    total_orders += 1
                    print('Estimated: {}'.format(order.time_in + order.prep_time + wait_time), 'Real: {}'.format(path.nodes[1:-1][order_idx].arrival_time))
                print('\n')
        print('Total orders deliverd: {}'.format(total_orders))
        
    def descriptive_stats(self, trial: int = 1, wait_time: int = 30):
        index_tuples = []
        data = []
        for rider_idx, rider in enumerate(self.riders):
            for path_idx, path in enumerate(rider.completed_paths):          
                deltas = []
                for order_idx, order in enumerate(rider.completed_orders[path_idx]):
                    delta = (path.nodes[1:-1][order_idx].arrival_time) - (order.time_in + order.prep_time + wait_time) # Real time delivered vs. estimated time.
                    deltas.append(delta)
                    
                path_time = path.completion_time - path.departure_time
                path_end_time = path.completion_time
                n_orders = len(rider.completed_orders[path_idx])
                delta_mean = sum(deltas)/len(deltas)
                delta_min = min(deltas)
                delta_max = max(deltas)
                
                data.append([path_time, path_end_time, n_orders, delta_mean, delta_min, delta_max])
                index_tuples.append(('trial {}'.format(trial), 'rider {}'.format(rider_idx + 1), 'path {}'.format(path_idx + 1)))
                
        index = pd.MultiIndex.from_tuples(index_tuples, names=['trial', 'rider', 'path'])
        df = pd.DataFrame(data, index=index, columns=['path_time', 'path_end_time', 'n_orders', 'delta_mean', 'delta_min', 'delta_max'])
        
        return df
    
    # [ UTILS END ]
    
    def update_routing(self, order: Order, time: int):       
        if order is not None: 
            if self.verbose: print('Updating routing...\nNew order {}'.format(order.id))
            self.orders.append(order)
        else:
            if self.verbose: print('Updating routing...\nNo new orders')
        
        free_riders = list(filter(lambda x: not x.on_path, self.riders))
        if len(free_riders) > 0:
            if self.verbose: print('Current path')

            time_matrix = RiderManager.time_matrix(self.orders)
            routes = RiderManager.vrp_path(time_matrix, len(free_riders))
        
            for (rider, route) in zip(free_riders, routes):
                if self.verbose: print('Rider {}, Route {}'.format(rider.name, route))
                if len(route) == 2: # No node assigned to route.
                    rider.current_path = None
                    rider.current_orders = []
                    continue

                path = Path(self.orders, route, time=time)
                if self.verbose: print('Departure time {}'.format(path.departure_time))                
                total_time = path.departure_time
                last_node = 0
                for (node, path_node) in zip(route, path.nodes):
                    total_time += time_matrix[last_node][node]
                    last_node = node
                    path_node.arrival_time = total_time
                path.completion_time = path.nodes[-1].arrival_time
            
                # Update rider's path
                rider.current_path = path
                rider.current_orders = [self.orders[x - 1] for x in route[1:-1]]
                
                # Remove order from next path
        #else:
        #    if self.verbose: print('Next path')
        #        
        #    time_matrix = RiderManager.time_matrix(self.orders)
        #    routes = RiderManager.vrp_path(time_matrix, len(self.riders))
        #    
        #    for (rider, route) in zip(self.riders, routes):
        #        if self.verbose: print('Rider {}, Route {}'.format(rider.name, route))
        #        if len(route) == 2: continue # No node assigned to route.
#
        #        path = Path(self.orders, route, previous_completion = rider.current_path.completion_time)
        #        if self.verbose: print('Completion time {} Departure time {}'.format(rider.current_path.completion_time, path.departure_time))
        #        total_time = path.departure_time
        #        last_node = 0
        #        for (node, path_node) in zip(route, path.nodes):
        #            total_time += time_matrix[last_node][node]
        #            last_node = node
        #            path_node.arrival_time = total_time
        #        path.completion_time = path.nodes[-1].arrival_time
        #    
        #        # Update rider's path
        #        rider.next_path = path
        #        rider.next_orders = [self.orders[x - 1] for x in route[1:-1]]
    
    def describe_update(self, rider):
        print('\nUpdating Rider {}'.format(rider.name))
        if rider.current_path is not None:
            print('On path: {}'.format(rider.on_path))
            print('Path len: {}'.format(len(rider.current_path.nodes)))
            print('Departure/Departed: {}'.format(rider.current_path.departure_time))
            print('Will return: {}'.format(rider.current_path.completion_time))
            print('Orders: {}'.format([x.id for x in rider.current_orders]))
        else:
            print('No paths for rider')
        
    def update(self, time):
        if self.verbose: print('Orders outstanding {}'.format(len(self.orders)))
        for rider in self.riders:
            if self.verbose: self.describe_update(rider)
                
            # Check if rider completed the path.
            if rider.on_path:
                if rider.current_path.completion_time == time:
                    rider.completed_paths.append(rider.current_path)
                    rider.completed_orders.append(rider.current_orders)
                    
                    # Update the rider's properties.
                    rider.current_path = rider.next_path
                    rider.next_path = None
                    rider.current_orders = rider.next_orders
                    rider.next_orders = []
                    rider.on_path = False
                    
                # Check if rider has to start the current path immediately.    
                if rider.current_path is None: continue
                if rider.current_path.departure_time == time:
                    rider.on_path = True
                    for order in rider.current_orders:
                        self.orders.remove(order)
            
                continue
            
            # Check if rider can start the path
            else:
                # Rider starts the path.
                if rider.current_path is None: continue
                if rider.current_path.departure_time == time:
                    rider.on_path = True
                    for order in rider.current_orders:
                        self.orders.remove(order)
    
            

In [13]:
class Simulator:    
    def __init__(self, num_orders: int, timeframe: int, num_riders: int, verbose: bool = False):
        self.time = 0
        self.num_orders = num_orders
        self.timeframe = timeframe
        self.verbose = verbose
        
        # Simulation specifics.
        self.orders = Simulator.generate_orders(self)
        
        # Rider manager.
        self.rider_manager = RiderManager(num_riders, self.time, self.verbose)
        
    # [ UTILS START ] 
    @staticmethod       
    def generate_orders(self) -> [Order]:
        orders = [Order(random.randrange(self.timeframe)) for x in range(self.num_orders)] # Can be improved with density function
        return orders
        
    @staticmethod       
    def blockPrint():
        sys.stdout = open(os.devnull, 'w')
    
    # Restore.
    @staticmethod  
    def enablePrint():
        sys.stdout = sys.__stdout__
    # [ UTILS END ] 
    
    def start(self):
        #if not verbose: Simulator.blockPrint()
        # Order times.
        while self.timeframe + 300 > self.time:
            if self.verbose: print('\n//// TIME {}'.format(self.time))
            # Update riders status.
            self.rider_manager.update(self.time)
            
            did_update = False
            for order in self.orders:
                if order.time_in == self.time: 
                    self.rider_manager.update_routing(order, self.time)
                    did_update = True
            if not did_update: self.rider_manager.update_routing(order=None, time=self.time)    
            
            self.time += 1
        #Simulator.enablePrint()
            
        

In [14]:
simulator = Simulator(13, 60, 3, verbose=True)

simulator.start()
simulator.rider_manager.orders_assigned
df = simulator.rider_manager.descriptive_stats()


//// TIME 0
Orders outstanding 0

Updating Rider R1
No paths for rider

Updating Rider R2
No paths for rider

Updating Rider R3
No paths for rider
Updating routing...
No new orders
Current path
Rider R1, Route [0, 0]
Rider R2, Route [0, 0]
Rider R3, Route [0, 0]

//// TIME 1
Orders outstanding 0

Updating Rider R1
No paths for rider

Updating Rider R2
No paths for rider

Updating Rider R3
No paths for rider
Updating routing...
No new orders
Current path
Rider R1, Route [0, 0]
Rider R2, Route [0, 0]
Rider R3, Route [0, 0]

//// TIME 2
Orders outstanding 0

Updating Rider R1
No paths for rider

Updating Rider R2
No paths for rider

Updating Rider R3
No paths for rider
Updating routing...
No new orders
Current path
Rider R1, Route [0, 0]
Rider R2, Route [0, 0]
Rider R3, Route [0, 0]

//// TIME 3
Orders outstanding 0

Updating Rider R1
No paths for rider

Updating Rider R2
No paths for rider

Updating Rider R3
No paths for rider
Updating routing...
No new orders
Current path
Rider R1, Rou

In [15]:
df

path_time  path_end_time  n_orders  delta_mean  \
trial   rider   path                                                     
trial 1 rider 1 path 1         26             52         3  -14.333333   
                path 2         38             98         3  -12.333333   
        rider 2 path 1         17             38         1  -18.000000   
                path 2         39             80         4   -8.750000   
        rider 3 path 1         27             47         1  -14.000000   
                path 2         23             71         1  -16.000000   

                        delta_min  delta_max  
trial   rider   path                          
trial 1 rider 1 path 1        -22         -8  
                path 2        -24         -1  
        rider 2 path 1        -18        -18  
                path 2        -20          1  
        rider 3 path 1        -14        -14  
                path 2        -16        -16

In [16]:
simulators = [Simulator(13, 60, 3) for x in range(1000)]

dfs = []
for idx, simulator in enumerate(simulators): 
    simulator.start()
    df = simulator.rider_manager.descriptive_stats(trial=idx+1)
    dfs.append(df)
    
    if idx == 0: print('Started')
    if (idx + 1)%100 == 0: print('{}%'.format((idx + 1)/10))

df = pd.concat(dfs)

Started
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100.0%


In [33]:
df

path_time  path_end_time  n_orders  delta_mean  \
trial      rider   path                                                     
trial 1    rider 1 path 1         13             20         1  -21.000000   
                   path 2         31             57         2  -11.500000   
                   path 3         21             84         1  -17.000000   
           rider 2 path 1         21             32         1  -17.000000   
                   path 2         13             49         1  -21.000000   
...                              ...            ...       ...         ...   
trial 1000 rider 2 path 1         15             34         1  -20.000000   
                   path 2         34             86         3   -4.333333   
           rider 3 path 1         11             24         1  -22.000000   
                   path 2         25             53         1  -15.000000   
                   path 3         35             99         3   -7.333333   

                           delta_min  delta_max  
trial      rider   path                          
trial 1    rider 1 path 1        -21        -21  
                   path 2        -17         -6  
                   path 3        -17        -17  
           rider 2 path 1        -17        -17  
                   path 2        -21        -21  
...                              ...        ...  
trial 1000 rider 2 path 1        -20        -20  
                   path 2        -11          7  
           rider 3 path 1        -22        -22  
                   path 2        -15        -15  
                   path 3        -17          5  

[7055 rows x 6 columns]

In [ ]:
df.to_csv('20220530_mc1000_riders

In [45]:
df.groupby(['trial', 'rider']).max().sort_values(by=['trial'], key=lambda col: col.str.higher())#.to_csv('test.csv')#.path_end_time.mean()

AttributeError: 'StringMethods' object has no attribute 'higher'

In [28]:
df.mean()

path_time        26.203969
path_end_time    59.075266
n_orders          1.840539
delta_mean      -12.951023
delta_min       -16.864352
delta_max        -9.030758
dtype: float64

In [29]:
df.to_excel("20220530_mc1000_riders3_orders13_timeframe60.xlsx", sheet_name='paths')  